In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

teamname = 'hd-group-unsw'
data_folder='s3://tf-trachack-data/212/'
root_folder='s3://tf-trachack-notebooks/'+teamname+'/jupyter/jovyan/'

# import all csv data
upgrades=pd.read_csv(data_folder+"data/dev/upgrades.csv")
customer_info=pd.read_csv(data_folder+"data/dev/customer_info.csv")
phone_info=pd.read_csv(data_folder+"data/dev/phone_info.csv")
redemptions=pd.read_csv(data_folder+"data/dev/redemptions.csv")
deactivations=pd.read_csv(data_folder+"data/dev/deactivations.csv")
reactivations=pd.read_csv(data_folder+"data/dev/reactivations.csv")
suspensions=pd.read_csv(data_folder+"data/dev/suspensions.csv")
network_usage_domestic=pd.read_csv(data_folder+"data/dev/network_usage_domestic.csv")
lrp_points=pd.read_csv(data_folder+"data/dev/lrp_points.csv")
lrp_enrollment=pd.read_csv(data_folder+"data/dev/lrp_enrollment.csv")

In [ ]:
# process upgrades and customer_info
up = upgrades.copy(deep = True)
up['upgrade'].replace({'yes':1,'no':0}, inplace = True)
customer = customer_info.copy(deep = True)
# transform time from str type to int type
first_activation_list = np.array(customer['first_activation_date'])
for i in range(len(first_activation_list)):
    if first_activation_list[i]==first_activation_list[i]:
        t = first_activation_list[i]
        t = time.strptime(t, "%Y-%M-%d")
        t = time.mktime(t)
        first_activation_list[i] = t
customer['first_activation_date'] = first_activation_list

redemption_list = np.array(customer['redemption_date'])
for i in range(len(redemption_list)):
    if redemption_list[i] == redemption_list[i]:
        t1 = redemption_list[i]
        t1 = time.strptime(t1, "%Y-%M-%d")
        t1 = time.mktime(t1)
        redemption_list[i] = t1
customer['redemption_date'] = redemption_list

# normalisation for time
max_v = np.max(customer.loc[:, 'first_activation_date'])
min_v = np.min(customer.loc[:, 'first_activation_date'])
customer.loc[:,'first_activation_date'] = (customer.loc[:,'first_activation_date']-min_v)/(max_v-min_v)

max_v = np.max(customer['redemption_date'])
min_v = np.min(customer['redemption_date'])
customer.loc[:,'redemption_date'] = (customer.loc[:,'redemption_date']-min_v)/(max_v-min_v)

customer['carrier'].replace({'carrier 1':1, 'carrier 2': 2, 'carrier 3': 3},inplace = True)
customer.drop(columns = 'plan_subtype',inplace = True)
# fill NAN with np.mean
customer['first_activation_date'].fillna(value=np.mean(customer['first_activation_date']), inplace=True)

customer['plan_name'].replace({'Other':0, 'plan 1':1, 'plan 2': 2, 'plan 3': 3,'plan 4':4},inplace = True )

customer['plan_name'].fillna(value=-1, inplace=True)
customer['redemption_date'].fillna(value=-1, inplace=True)
# merge two tables
train_data = pd.merge(up,customer,how='inner',on='line_id')

In [ ]:
# select some features from phone_info
# transform data into numeric and process NAN
selected_phone_info = phone_info.loc[:,['line_id','cpu_cores','expandable_storage','gsma_device_type','lte'
                                        ,'lte_advanced','os_name','sim_size','touch_screen','wi_fi','year_released']]
selected_phone_info['cpu_cores'].replace({'1+2+4':1,'1':2,'2+2+4':3,'4+2':4,'1+3+4':5,'2+2':6,
                          '6+2':7,'2+6':8,'4+4':9,'8':10,'2+4':11,'2':12,'4':13}, inplace = True)
selected_phone_info['gsma_device_type'].replace({'Mobile Phone/Feature phone':1,'Handheld':2, 'Smartphone':3,'WLAN Router':4, 'Modem': 5, 'Tablet': 6}, inplace = True)
selected_phone_info['os_name'].replace({'Windows Phone':1,'Android':2, 'iOS':3,'KaiOS':4, 'LG proprietary': 5, 'Nucleus': 6,'Samsung proprietary':7,'Nokia OS':8,'Other':9}, inplace = True)
selected_phone_info['sim_size'].replace({'Micro & Mini':1,'Micro & Nano':2,'Micro':3,'Nano':4,'Mini': 5}, inplace = True)
selected_phone_info['cpu_cores'].fillna(value=-1, inplace=True)
selected_phone_info['expandable_storage'].fillna(value=-1, inplace=True)
selected_phone_info['gsma_device_type'].fillna(value=-1, inplace=True)
selected_phone_info['lte'].fillna(value=-1, inplace=True)
selected_phone_info['lte_advanced'].fillna(value=-1, inplace=True)
selected_phone_info['os_name'].fillna(value=-1, inplace=True)
selected_phone_info['sim_size'].fillna(value=-1, inplace=True)
selected_phone_info['touch_screen'].fillna(value=-1, inplace=True)
selected_phone_info['wi_fi'].fillna(value=-1, inplace=True)
selected_phone_info['year_released'].fillna(value=-1, inplace=True)
train_data = pd.merge(train_data,selected_phone_info,how='inner',on='line_id')

In [ ]:
# process network_usage_domestic
network = network_usage_domestic.loc[:,['line_id','mms_in','mms_out','sms_in','sms_out','total_kb','voice_count_total']]
network = network.groupby('line_id').sum()
train_data = pd.merge(train_data,network,on='line_id',how='left')
max_v = np.max(train_data.loc[:, 'total_kb'])
min_v = np.min(train_data.loc[:, 'total_kb'])
train_data.loc[:,'total_kb'] = (train_data.loc[:,'total_kb']-min_v)/(max_v-min_v)
train_data['mms_in'].fillna(value=-1, inplace=True)
train_data['mms_out'].fillna(value=-1, inplace=True)
train_data['sms_in'].fillna(value=-1, inplace=True)
train_data['sms_out'].fillna(value=-1, inplace=True)
train_data['total_kb'].fillna(value=-1, inplace=True)
train_data['voice_count_total'].fillna(value=-1, inplace=True)

In [ ]:
# process redemption table
# picking redemption_times and gross_revenue as features
redem = redemptions.groupby('line_id').sum()
redem.reset_index()
redemption_times = []
id_times = {}
times = redemptions['line_id'].value_counts()
userid = times.index.tolist()
for i in range(len(userid)):
    id_times[userid[i]] = times[userid[i]]
userid_redemptions = upgrades['line_id'].tolist()
for i in range(len(userid_redemptions)):
    if userid_redemptions[i] in id_times.keys():
        redemption_times.append(times[userid_redemptions[i]])
    else:
        redemption_times.append(0)
        
train_data = pd.merge(train_data, redem, on='line_id',how='left')
train_data['redemption_times']=redemption_times
train_data['gross_revenue'].fillna(value = 0,inplace=True)

In [ ]:
# deactivation table
line_id = upgrades['line_id'].tolist()
line_id_deact = set(deactivations['line_id'].tolist())
latest_deactivation_date = []
latest_deactivation_date_dict = {}
deact_times = []
deact_times_dict={}
# obtain times and latest_deactivation_date
for i in range(len(line_id)):
    if line_id[i] in line_id_deact:
        info = deactivations[deactivations['line_id']==line_id[i]]
        date = info['deactivation_date'].tolist()
        for j in range(len(date)):
            t = time.strptime(date[j], "%Y-%M-%d")
            t = time.mktime(t)
            date[j] = t
        latest_deactivation_date.append(np.max(date))
        deact_times.append(len(date))
    else:
        latest_deactivation_date.append(0)
        deact_times.append(0)

train_data['latest_deactivation_date'] = latest_deactivation_date
train_data['deact_times'] = deact_times
# normalisation
max_v = np.max(train_data.loc[:, 'latest_deactivation_date'])
min_v = np.min(train_data.loc[:, 'latest_deactivation_date'])
train_data.loc[:,'latest_deactivation_date'] = (train_data.loc[:,'latest_deactivation_date']-min_v)/(max_v-min_v)
# one-hot encoder for deactivation_reason
deact_dummy = pd.get_dummies(deactivations,columns=['deactivation_reason'])
deact_dummy = deact_dummy.loc[:,['line_id','deactivation_date','deactivation_reason_ACTIVE UPGRADE','deactivation_reason_UPGRADE','deactivation_reason_PASTDUE']]
deact_dummy = deact_dummy.groupby('line_id')['deactivation_reason_ACTIVE UPGRADE','deactivation_reason_UPGRADE','deactivation_reason_PASTDUE'].sum().reset_index()
train_data = pd.merge(train_data, deact_dummy, on='line_id',how='left')
train_data['deactivation_reason_ACTIVE UPGRADE'].fillna(value = 0, inplace = True)
train_data['deactivation_reason_UPGRADE'].fillna(value=0, inplace=True)
train_data['deactivation_reason_PASTDUE'].fillna(value=0, inplace=True)

In [ ]:
# reacitivation
line_id = upgrades['line_id'].tolist()
line_id_deact = set(reactivations['line_id'].tolist())
latest_reactivations_date = []
react_times = []

# obtain times and latest_reactivations_date
for i in range(len(line_id)):
    if line_id[i] in line_id_deact:
        info = reactivations[reactivations['line_id']==line_id[i]]
        date = info['reactivation_date'].tolist()
        for j in range(len(date)):
            t = time.strptime(date[j], "%Y-%M-%d")
            t = time.mktime(t)
            date[j] = t
        latest_reactivations_date.append(np.max(date))
        react_times.append(len(date))
    else:
        latest_reactivations_date.append(0)
        react_times.append(0)
# normalisation
train_data['latest_reactivations_date'] = latest_reactivations_date
train_data['react_times'] = react_times
max_v = np.max(train_data.loc[:, 'latest_reactivations_date'])
min_v = np.min(train_data.loc[:, 'latest_reactivations_date'])
train_data.loc[:,'latest_reactivations_date'] = (train_data.loc[:,'latest_reactivations_date']-min_v)/(max_v-min_v)

# suspensions
line_id = upgrades['line_id'].tolist()
line_id_sus = set(suspensions['line_id'].tolist())
sus_times = []

# obtain suspension times
for i in range(len(line_id)):
    if line_id[i] in line_id_sus:
        info = suspensions[suspensions['line_id']==line_id[i]]
        date = info['suspension_start_date'].tolist()
        sus_times.append(len(date))
    else:
        sus_times.append(0)
train_data['sus_times'] = sus_times


In [ ]:
# lrp_points
line_id = upgrades['line_id'].tolist()
line_id_lrp = set(lrp_points['line_id'].tolist())
total_quantity = []
lrp_points.dropna(axis = 0, subset = ['total_quantity'], inplace = True)

for i in range(len(line_id)):
    if line_id[i] in line_id_lrp:
        info = lrp_points[lrp_points['line_id']==line_id[i]]
        total_quantity.append(info['total_quantity'].tolist()[0])
    else:
        total_quantity.append(0)

train_data['total_quantity'] = total_quantity

# lrp_enrolments
lrp_enrollment
line_id_lrp_enrol = lrp_enrollment['line_id'].tolist()
line_id_lrp_enrol = list(set(line_id_lrp_enrol))
lrp_enrolled = []
for i in range(len(line_id)):
    if line_id[i] in line_id_lrp_enrol:
        lrp_enrolled.append(1)
    else:
        lrp_enrolled.append(0)
        
train_data['lrp_enrolled'] = lrp_enrolled

In [ ]:
# train test split for training dataset
df_Y = train_data['upgrade']
df_X = train_data.drop(columns=['line_id','date_observed','upgrade'])
X_train, X_test, Y_train, Y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=511)

In [ ]:
# import eval tables
upgrades_eval=pd.read_csv(data_folder+"data/eval/upgrades.csv")
customer_info_eval=pd.read_csv(data_folder+"data/eval/customer_info.csv")
phone_info_eval=pd.read_csv(data_folder+"data/eval/phone_info.csv")
redemptions_eval=pd.read_csv(data_folder+"data/eval/redemptions.csv")
deactivations_eval=pd.read_csv(data_folder+"data/eval/deactivations.csv")
reactivations_eval=pd.read_csv(data_folder+"data/eval/reactivations.csv")
suspensions_eval=pd.read_csv(data_folder+"data/eval/suspensions.csv")
network_usage_domestic_eval=pd.read_csv(data_folder+"data/eval/network_usage_domestic.csv")
lrp_points_eval=pd.read_csv(data_folder+"data/eval/lrp_points.csv")
lrp_enrollment_eval=pd.read_csv(data_folder+"data/eval/lrp_enrollment.csv")

In [ ]:
# do the same process as training data
# eval data
up_eval = upgrades_eval.copy(deep = True)

customer_eval = customer_info_eval.copy(deep = True)
# 将str时间类型 改为float类型
first_activation_list = np.array(customer_eval['first_activation_date'])
for i in range(len(first_activation_list)):
    if first_activation_list[i]==first_activation_list[i]:
        t = first_activation_list[i]
        t = time.strptime(t, "%Y-%M-%d")
        t = time.mktime(t)
        first_activation_list[i] = t
customer_eval['first_activation_date'] = first_activation_list

redemption_list = np.array(customer_eval['redemption_date'])
for i in range(len(redemption_list)):
    if redemption_list[i] == redemption_list[i]:
        t1 = redemption_list[i]
        t1 = time.strptime(t1, "%Y-%M-%d")
        t1 = time.mktime(t1)
        redemption_list[i] = t1
customer_eval['redemption_date'] = redemption_list

# 归一化
max_v = np.max(customer_eval.loc[:, 'first_activation_date'])
min_v = np.min(customer_eval.loc[:, 'first_activation_date'])
customer_eval.loc[:,'first_activation_date'] = (customer_eval.loc[:,'first_activation_date']-min_v)/(max_v-min_v)

max_v = np.max(customer_eval['redemption_date'])
min_v = np.min(customer_eval['redemption_date'])
customer_eval.loc[:,'redemption_date'] = (customer_eval.loc[:,'redemption_date']-min_v)/(max_v-min_v)

customer_eval['carrier'].replace({'carrier 1':1, 'carrier 2': 2, 'carrier 3': 3},inplace = True)
customer_eval.drop(columns = 'plan_subtype',inplace = True)
customer_eval['first_activation_date'].fillna(value=np.mean(customer_eval['first_activation_date']), inplace=True)
customer_eval['plan_name'].replace({'Other':0, 'plan 1':1, 'plan 2': 2, 'plan 3': 3,'plan 4':4},inplace = True )
customer_eval['plan_name'].fillna(value=-1, inplace=True)
customer_eval['redemption_date'].fillna(value=-1, inplace=True)

eval_data = pd.merge(up_eval,customer_eval,how='inner',on='line_id')

selected_phone_info = phone_info_eval.loc[:,['line_id','cpu_cores','expandable_storage','gsma_device_type','lte'
                                        ,'lte_advanced','os_name','sim_size','touch_screen','wi_fi','year_released']]
selected_phone_info['cpu_cores'].replace({'1+2+4':1,'1':2,'2+2+4':3,'4+2':4,'1+3+4':5,'2+2':6,
                          '6+2':7,'2+6':8,'4+4':9,'8':10,'2+4':11,'2':12,'4':13}, inplace = True)
selected_phone_info['gsma_device_type'].replace({'Mobile Phone/Feature phone':1,'Handheld':2, 'Smartphone':3,'WLAN Router':4, 'Modem': 5, 'Tablet': 6}, inplace = True)
selected_phone_info['os_name'].replace({'Windows Phone':1,'Android':2, 'iOS':3,'KaiOS':4, 'LG proprietary': 5, 'Nucleus': 6,'Samsung proprietary':7,'Nokia OS':8,'Other':9}, inplace = True)
selected_phone_info['sim_size'].replace({'Micro & Mini':1,'Micro & Nano':2,'Micro':3,'Nano':4,'Mini': 5}, inplace = True)
selected_phone_info['cpu_cores'].fillna(value=-1, inplace=True)
selected_phone_info['expandable_storage'].fillna(value=-1, inplace=True)
selected_phone_info['gsma_device_type'].fillna(value=-1, inplace=True)
selected_phone_info['lte'].fillna(value=-1, inplace=True)
selected_phone_info['lte_advanced'].fillna(value=-1, inplace=True)
selected_phone_info['os_name'].fillna(value=-1, inplace=True)
selected_phone_info['sim_size'].fillna(value=-1, inplace=True)
selected_phone_info['touch_screen'].fillna(value=-1, inplace=True)
selected_phone_info['wi_fi'].fillna(value=-1, inplace=True)
selected_phone_info['year_released'].fillna(value=-1, inplace=True)

eval_data = pd.merge(eval_data,selected_phone_info,how='inner',on='line_id')

network_eval = network_usage_domestic_eval.loc[:,['line_id','mms_in','mms_out','sms_in','sms_out','total_kb','voice_count_total']]
network_eval = network_eval.groupby('line_id').sum()
eval_data = pd.merge(eval_data,network_eval,on='line_id',how='left')
max_v = np.max(eval_data.loc[:, 'total_kb'])
min_v = np.min(eval_data.loc[:, 'total_kb'])
eval_data.loc[:,'total_kb'] = (eval_data.loc[:,'total_kb']-min_v)/(max_v-min_v)
eval_data['mms_in'].fillna(value=-1, inplace=True)
eval_data['mms_out'].fillna(value=-1, inplace=True)
eval_data['sms_in'].fillna(value=-1, inplace=True)
eval_data['sms_out'].fillna(value=-1, inplace=True)
eval_data['total_kb'].fillna(value=-1, inplace=True)
eval_data['voice_count_total'].fillna(value=-1, inplace=True)

redem = redemptions_eval.groupby('line_id').sum()
redem.reset_index()
redemption_times = []
id_times = {}
times = redemptions_eval['line_id'].value_counts()
userid = times.index.tolist()
for i in range(len(userid)):
    id_times[userid[i]] = times[userid[i]]
userid_redemptions = upgrades_eval['line_id'].tolist()
for i in range(len(userid_redemptions)):
    if userid_redemptions[i] in id_times.keys():
        redemption_times.append(times[userid_redemptions[i]])
    else:
        redemption_times.append(0)
eval_data = pd.merge(eval_data, redem, on='line_id',how='left')
eval_data['redemption_times']=redemption_times
eval_data['gross_revenue'].fillna(value = 0,inplace=True)

# deactivation
line_id = upgrades_eval['line_id'].tolist()
line_id_deact = set(deactivations_eval['line_id'].tolist())
latest_deactivation_date = []

deact_times = []
for i in range(len(line_id)):
    if line_id[i] in line_id_deact:
        info = deactivations_eval[deactivations_eval['line_id']==line_id[i]]
        date = info['deactivation_date'].tolist()
        for j in range(len(date)):
            t = time.strptime(date[j], "%Y-%M-%d")
            t = time.mktime(t)
            date[j] = t
        latest_deactivation_date.append(np.max(date))
        deact_times.append(len(date))
    else:
        latest_deactivation_date.append(0)
        deact_times.append(0)
        
eval_data['latest_deactivation_date'] = latest_deactivation_date
eval_data['deact_times'] = deact_times

max_v = np.max(eval_data.loc[:, 'latest_deactivation_date'])
min_v = np.min(eval_data.loc[:, 'latest_deactivation_date'])
eval_data.loc[:,'latest_deactivation_date'] = (eval_data.loc[:,'latest_deactivation_date']-min_v)/(max_v-min_v)
deact_dummy = pd.get_dummies(deactivations_eval,columns=['deactivation_reason'])
deact_dummy = deact_dummy.loc[:,['line_id','deactivation_date','deactivation_reason_ACTIVE UPGRADE','deactivation_reason_UPGRADE','deactivation_reason_PASTDUE']]
deact_dummy = deact_dummy.groupby('line_id')['deactivation_reason_ACTIVE UPGRADE','deactivation_reason_UPGRADE','deactivation_reason_PASTDUE'].sum().reset_index()
eval_data = pd.merge(eval_data, deact_dummy, on='line_id',how='left')
eval_data['deactivation_reason_ACTIVE UPGRADE'].fillna(value = 0, inplace = True)
eval_data['deactivation_reason_UPGRADE'].fillna(value=0, inplace=True)
eval_data['deactivation_reason_PASTDUE'].fillna(value=0, inplace=True)

# reactivation
line_id_deact = set(reactivations_eval['line_id'].tolist())
latest_reactivations_date = []
react_times = []


for i in range(len(line_id)):
    if line_id[i] in line_id_deact:
        info = reactivations_eval[reactivations_eval['line_id']==line_id[i]]
        date = info['reactivation_date'].tolist()
        for j in range(len(date)):
            t = time.strptime(date[j], "%Y-%M-%d")
            t = time.mktime(t)
            date[j] = t
        latest_reactivations_date.append(np.max(date))
        react_times.append(len(date))
    else:
        latest_reactivations_date.append(0)
        react_times.append(0)

eval_data['latest_reactivations_date'] = latest_reactivations_date
eval_data['react_times'] = react_times        
max_v = np.max(eval_data.loc[:, 'latest_reactivations_date'])
min_v = np.min(eval_data.loc[:, 'latest_reactivations_date'])
eval_data.loc[:,'latest_reactivations_date'] = (eval_data.loc[:,'latest_reactivations_date']-min_v)/(max_v-min_v)

# suspensions
line_id_sus = set(suspensions_eval['line_id'].tolist())
sus_times = []


for i in range(len(line_id)):
    if line_id[i] in line_id_sus:
        info = suspensions_eval[suspensions_eval['line_id']==line_id[i]]
        date = info['suspension_start_date'].tolist()
        sus_times.append(len(date))
    else:
        sus_times.append(0)
        
eval_data['sus_times'] = sus_times

# lrp_points
lrp_eval = lrp_points_eval.dropna(axis = 0, subset = ['total_quantity'])
line_id_lrp = set(lrp_eval['line_id'].tolist())
total_quantity = []

for i in range(len(line_id)):
    if line_id[i] in line_id_lrp:
        info = lrp_eval[lrp_eval['line_id']==line_id[i]]
        total_quantity.append(info['total_quantity'].tolist()[0])
    else:
        total_quantity.append(0)

eval_data['total_quantity'] = total_quantity

line_id_lrp_enrol = lrp_enrollment_eval['line_id'].tolist()
line_id_lrp_enrol = list(set(line_id_lrp_enrol))
lrp_enrolled = []
for i in range(len(line_id)):
    if line_id[i] in line_id_lrp_enrol:
        lrp_enrolled.append(1)
    else:
        lrp_enrolled.append(0)
eval_data['lrp_enrolled'] = lrp_enrolled


In [ ]:
eval_features = eval_data.drop(columns=['line_id','date_observed'])

In [ ]:
# please pip install xgboost befor using this module
import xgboost as xgb
model_xgb = xgb.XGBClassifier(n_estimators = 6000,
                              learning_rate=0.01,
                              max_depth=10, 
                              min_child_weight=1,
                              subsample=0.9,
                              gamma=0.2,
                              colsample_bytree=0.7,
                              objective= 'binary:logistic',
                              nthread=4,
                              scale_pos_weight=1,
                              seed=27,
                              reg_alpha=1e-05,
                              use_label_encoder = False)
model_xgb.fit(df_X, df_Y)

In [ ]:
eval_prediction = model_xgb.predict(eval_features)

In [ ]:
eval_id = upgrades_eval['line_id'].tolist()
predictions=pd.DataFrame(eval_id,columns=['line_id'])
predictions['prediction']=eval_prediction
submission_path=root_folder+"submission/2021-04-25.csv"
predictions.to_csv(submission_path,header=True,index=None)